In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE
rf = RandomForestClassifier()
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from scipy.stats import randint

In [2]:
# data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD.csv')
# data.index = data['PatientID']

In [7]:
data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD_90percent_sample.csv')
data.index = data['PatientID']
# drop the 'PatientID' column
data = data.drop('PatientID', axis=1)

In [8]:
data_target = pd.read_csv('~/Desktop/project_data_new/target_768_avg_expanded.csv')

In [9]:
data_target.index = data_target['Unnamed: 0']
data_target = data_target.drop(['Unnamed: 0'], axis = 1)
# only keep the columns with category in the name
data_target = data_target.loc[:, data_target.columns.str.contains('category')]

In [13]:
data = data[data.index.isin(data_target.index)]

In [52]:
data

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
PatientID,,,,,,,,,,,,,,,,,,,,,
TCGA-A6-6650,-0.457220,0.851885,0.540813,-0.423622,0.714115,-0.564329,1.607602,0.543431,-0.870704,0.980508,...,0.673500,0.445102,0.272053,0.577840,-0.313309,0.463409,0.216033,1.875605,-0.811024,-1.250726
TCGA-5M-AAT4,-0.430112,0.766393,0.768542,-0.344121,0.560576,-0.597754,1.708495,0.419003,-0.934103,1.152953,...,0.578889,0.539531,0.399675,0.471158,-0.089670,0.701613,0.178599,1.802830,-0.844305,-1.300183
TCGA-A6-2676,-0.417006,0.964753,0.525541,-0.572356,0.777732,-0.462124,1.476457,0.640170,-0.722137,1.066126,...,0.475959,0.557490,0.094860,0.659872,-0.243587,0.122108,0.107965,2.146043,-0.946326,-1.345658
TCGA-A6-6652,-0.276354,0.766071,0.612194,-0.596442,0.814464,0.016801,1.598807,0.929635,-0.914753,1.071625,...,0.395806,0.545849,0.290805,0.229333,-0.536335,0.017757,-0.074384,2.026599,-0.776906,-0.934953
TCGA-A6-6651,-0.341245,0.842662,0.646159,-0.583673,0.831014,-0.383301,1.613733,0.694289,-0.983706,1.084958,...,0.489947,0.634749,0.419969,0.541244,-0.382445,0.121604,0.071052,2.075633,-0.826747,-1.235763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-G4-6321,-0.359843,0.929950,0.543955,-0.350527,0.404216,-0.566966,1.688820,0.196331,-1.050643,1.162059,...,0.802867,0.499934,0.404179,0.325667,-0.166769,0.612408,0.197617,1.686868,-0.917694,-1.251037
TCGA-RU-A8FL,-0.480253,0.931134,0.455165,-0.714430,0.685353,-0.373306,1.511218,0.747226,-0.652520,0.907837,...,0.179997,0.744851,0.062442,0.249245,-0.573766,-0.159084,-0.157464,2.173445,-0.916892,-1.098334
TCGA-G4-6303,-0.397405,0.793659,0.682602,-0.526544,0.733674,-0.456034,1.593480,0.500975,-0.982290,1.068642,...,0.514841,0.515760,0.324934,0.458768,-0.172873,0.498273,0.168238,1.823696,-0.805411,-1.238766


In [14]:
data_test = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD_10percent_sample.csv')
data_test.index = data_test['PatientID']
# drop the 'PatientID' column
data_test = data_test.drop('PatientID', axis=1)
data_test = data_test[data_test.index.isin(data_target.index)]
data_test

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
PatientID,,,,,,,,,,,,,,,,,,,,,
TCGA-CM-6171,-0.446768,0.867309,0.546879,-0.416589,0.751486,-0.488991,1.547392,0.656082,-0.952199,0.940552,...,0.575794,0.498878,0.307701,0.433799,-0.283791,0.360137,0.158178,1.919735,-0.731380,-1.331179
TCGA-AA-3842,-0.285634,0.078604,0.821238,-0.694562,0.514553,0.122391,1.520501,0.551521,-0.909149,0.952024,...,0.039089,0.877942,1.164211,0.675850,-0.339191,-0.111566,0.134048,2.010713,-0.565601,-1.020457
TCGA-AA-3561,-0.343345,0.522357,0.749190,-0.579889,0.410007,-0.242321,1.746218,0.415922,-0.962094,1.111298,...,0.645423,0.679194,0.766653,0.497430,-0.308955,0.451458,0.090810,2.016258,-0.726342,-1.140222
TCGA-G4-6315,-0.302234,0.872818,0.812011,-0.382311,0.660013,-0.575302,1.479122,0.305756,-0.995347,1.249228,...,0.545346,0.549893,0.270967,0.533808,-0.071733,0.761300,0.183157,1.781668,-0.904719,-1.424805
TCGA-AY-4071,-0.309921,0.617729,0.654866,-0.860937,0.629124,-0.103098,1.212532,0.640801,-0.634122,0.883671,...,0.071015,0.817352,0.269195,0.528159,-0.363689,-0.342890,-0.111240,2.367249,-0.781699,-1.033974
TCGA-AA-A00L,-0.233838,0.063693,0.940162,-0.551396,0.429682,0.157665,1.626283,0.348518,-0.954313,0.859834,...,0.258132,0.864870,1.261963,0.703379,-0.414424,-0.002204,0.262086,1.871945,-0.459332,-1.006865
TCGA-D5-6535,-0.331603,0.570989,0.679557,-0.385874,0.518904,-0.401544,1.648816,0.388518,-1.188542,1.201107,...,0.888883,0.658257,0.684829,0.409407,-0.193154,0.468114,0.200221,1.825918,-0.739200,-1.254180
TCGA-AZ-6605,-0.425853,1.009269,0.696068,-0.430870,0.717048,-0.558006,1.623487,0.412603,-0.738047,1.277100,...,0.420962,0.465787,0.052568,0.563185,-0.089327,0.603820,0.050743,1.931762,-0.898702,-1.366013
TCGA-AA-3814,-0.436531,0.558981,0.860228,-0.850539,0.670436,-0.203063,1.272505,0.581573,-0.668134,1.039619,...,0.098623,0.805051,0.343421,0.765543,-0.203149,0.184580,-0.001479,2.108348,-0.803701,-1.162500


In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_selection import RFE
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
# from scipy.stats import randint

# # Initialize the Random Forest Classifier
# rf = RandomForestClassifier(random_state=42)

# # Extract the feature values from data
# X = data.values

# # Set the chunk size (number of target variables per part)
# chunk_size = 40

# # Loop through target columns in chunks
# for chunk_start in range(120, 241, chunk_size):
#     # Get the current chunk of target columns
#     chunk_end = min(chunk_start + chunk_size, 241)
#     target_chunk = data_target.columns[chunk_start:chunk_end]
    
#     # Initialize list to store results for the current chunk
#     chunk_results = []

#     # Open a file to write classification reports for the current chunk
#     report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Classification_Reports_100features_part_{chunk_start}_{chunk_end}.txt"
#     with open(report_file_path, "w") as report_file:
#         #stratify 
#         for target_col in target_chunk:
#             # Extract the target column for the current category
#             Y = data_target[target_col].values

#             # Select top 100 features using RFE
#             rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
#             X_selected = rfe.fit_transform(X, Y)
            
#             # Split the data into training and testing sets
#             #use stratify to cross validate
            
#             X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size=0.2, random_state=42)

#             # Define the hyperparameter distribution
#             param_dist = {
#                 'n_estimators': randint(100, 200),
#                 'max_depth': randint(20, 40),
#                 'min_samples_split': randint(10, 20),
#                 'min_samples_leaf': randint(1, 5),
#                 'max_features': ['sqrt', 'log2']
#             }

#             # Perform Randomized Search with cross-validation
#             random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=60, 
#                                                cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
#             random_search.fit(X_train, Y_train)

#             # Get the best model from Randomized Search
#             best_rf = random_search.best_estimator_
#             best_params = random_search.best_params_

#             # Fit the model on the training data for evaluation
#             best_rf.fit(X_train, Y_train)

#             # Evaluate on the training set
#             Y_train_pred = best_rf.predict(X_train)
#             train_report = classification_report(Y_train, Y_train_pred)
            
#             # Evaluate on the test set
#             Y_test_pred = best_rf.predict(X_test)
#             test_report = classification_report(Y_test, Y_test_pred, output_dict=True)

#             # Write classification reports to the single file
#             report_file.write(f"Classification Report for {target_col} (Train Set):\n")
#             report_file.write(train_report)
#             report_file.write("\n")
#             report_file.write(f"Classification Report for {target_col} (Test Set):\n")
#             report_file.write(classification_report(Y_test, Y_test_pred))
#             report_file.write("\n\n" + "="*80 + "\n\n")

#             # Calculate and store precision, recall, and accuracy
#             precision = precision_score(Y_test, Y_test_pred, average='weighted')
#             recall = recall_score(Y_test, Y_test_pred, average='weighted')
#             accuracy = accuracy_score(Y_test, Y_test_pred)

#             # Extract precision, recall, and f1-score specifically for class '1'
#             class_1_metrics = test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

#             chunk_results.append({
#                 "Target Variable": target_col,
#                 "Precision": precision,
#                 "Recall": recall,
#                 "Accuracy": accuracy,
#                 "Class 1 Precision": class_1_metrics["precision"],
#                 "Class 1 Recall": class_1_metrics["recall"],
#                 "Class 1 F1-Score": class_1_metrics["f1-score"],
#                 "Best Hyperparameters": best_params
#             })
    
#     # Save the results for the current chunk to an Excel file
#     results_df = pd.DataFrame(chunk_results)
#     results_df.to_excel(f"/home/qiuaodon/Desktop/CRC_image/Best_features_REF/Precision_Recall_Accuracy_100features_part_{chunk_start}_{chunk_end}.xlsx", index=False)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [15]:
# Align the targets for training and unseen test sets
data_target_train = data_target.loc[data.index]
data_target_unseen = data_target.loc[data_test.index]
data_target_train

,category_tnk_1,category_tnk_2,category_tnk_3,category_tnk_4,category_tnk_5,category_tnk_6,category_tnk_7,category_tnk_8,category_tnk_9,category_tnk_10,...,category_stromal_62,category_stromal_63,category_stromal_64,category_stromal_65,category_stromal_66,category_stromal_68,category_stromal_73,category_stromal_78,category_stromal_80,category_stromal_85
PatientID,,,,,,,,,,,,,,,,,,,,,
TCGA-A6-6650,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-5M-AAT4,-1,-1,-1,0,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-A6-2676,1,1,1,0,1,1,1,1,1,1,...,0,-1,0,-1,0,1,0,1,1,1
TCGA-A6-6652,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
TCGA-A6-6651,1,1,1,-1,1,1,1,1,-1,1,...,1,1,1,1,1,-1,1,1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-G4-6321,1,1,1,-1,1,1,1,1,-1,1,...,0,0,-1,1,1,-1,1,1,-1,-1
TCGA-RU-A8FL,-1,-1,-1,1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-G4-6303,-1,0,-1,-1,0,0,0,-1,-1,1,...,1,1,1,0,1,-1,0,0,-1,-1


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from scipy.stats import randint

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Extract the feature values from data
X = data.values
X_test_unseen = data_test.values
# Set the chunk size (number of target variables per part)
chunk_size = 40

for chunk_start in range(200, 241, chunk_size):
    # Get the current chunk of target columns
    chunk_end = min(chunk_start + chunk_size, 241)
    target_chunk = data_target.columns[chunk_start:chunk_end]
    
    # Initialize list to store results for the current chunk
    chunk_results = []

    # Open a file to write classification reports for the current chunk
    report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Classification_Reports_100features_part_{chunk_start}_{chunk_end}.txt"
    with open(report_file_path, "w") as report_file:
        for target_col in target_chunk:
            # Extract the target column for the current category from training/validation set
            Y = data_target_train[target_col].values

            # Select top 100 features using RFE
            rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
            X_selected = rfe.fit_transform(X, Y)

            # Split the remaining training/validation set into train and validation
            X_train, X_val, Y_train, Y_val = train_test_split(X_selected, Y, test_size=0.2, random_state=42, stratify=Y)

            # Define the hyperparameter distribution
            param_dist = {
                'n_estimators': randint(100, 200),
                'max_depth': randint(20, 40),
                'min_samples_split': randint(10, 20),
                'min_samples_leaf': randint(1, 5),
                'max_features': ['sqrt', 'log2']
            }

            # Perform Randomized Search with cross-validation
            random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=60, 
                                               cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
            random_search.fit(X_train, Y_train)

            # Get the best model from Randomized Search
            best_rf = random_search.best_estimator_
            best_params = random_search.best_params_

            # Fit the model on the training data for evaluation
            best_rf.fit(X_train, Y_train)

            # Evaluate on the training set
            Y_train_pred = best_rf.predict(X_train)
            train_report = classification_report(Y_train, Y_train_pred)
            
            # Evaluate on the validation set
            Y_val_pred = best_rf.predict(X_val)
            val_report = classification_report(Y_val, Y_val_pred, output_dict=True)

            # Test on the unseen test data
            X_test_unseen_selected = rfe.transform(X_test_unseen)  # Transform unseen test set using fitted RFE
            Y_test_unseen_col = data_target_unseen[target_col].values
            Y_test_pred = best_rf.predict(X_test_unseen_selected)
            unseen_test_report = classification_report(Y_test_unseen_col, Y_test_pred, output_dict=True)

            # Write classification reports to the single file
            report_file.write(f"Classification Report for {target_col} (Train Set):\n")
            report_file.write(train_report)
            report_file.write("\n")
            report_file.write(f"Classification Report for {target_col} (Validation Set):\n")
            report_file.write(classification_report(Y_val, Y_val_pred))
            report_file.write("\n")
            report_file.write(f"Classification Report for {target_col} (Unseen Test Set):\n")
            report_file.write(classification_report(Y_test_unseen_col, Y_test_pred))
            report_file.write("\n\n" + "="*80 + "\n\n")

            # Calculate and store metrics for validation set
            precision = precision_score(Y_val, Y_val_pred, average='weighted')
            recall = recall_score(Y_val, Y_val_pred, average='weighted')
            accuracy = accuracy_score(Y_val, Y_val_pred)

            # Extract validation Class 1 metrics
            class_1_metrics = val_report.get('1', {"precision": None, "recall": None, "f1-score": None})

            # Extract unseen test Class 1 metrics
            class_1_unseen_metrics = unseen_test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

            # Calculate unseen test metrics
            unseen_precision = precision_score(Y_test_unseen_col, Y_test_pred, average='weighted')
            unseen_recall = recall_score(Y_test_unseen_col, Y_test_pred, average='weighted')
            unseen_accuracy = accuracy_score(Y_test_unseen_col, Y_test_pred)

            # Append results for the current target variable
            chunk_results.append({
                "Target Variable": target_col,
                "Validation Precision": precision,
                "Validation Recall": recall,
                "Validation Accuracy": accuracy,
                "Class 1 Precision (Validation)": class_1_metrics["precision"],
                "Class 1 Recall (Validation)": class_1_metrics["recall"],
                "Class 1 F1-Score (Validation)": class_1_metrics["f1-score"],
                "Unseen Test Precision": unseen_precision,
                "Unseen Test Recall": unseen_recall,
                "Unseen Test Accuracy": unseen_accuracy,
                "Class 1 Precision (Unseen Test)": class_1_unseen_metrics["precision"],
                "Class 1 Recall (Unseen Test)": class_1_unseen_metrics["recall"],
                "Class 1 F1-Score (Unseen Test)": class_1_unseen_metrics["f1-score"],
                "Best Hyperparameters": best_params
            })
    
    # Save the results for the current chunk to an Excel file
    results_df = pd.DataFrame(chunk_results)
    results_df.to_excel(f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Precision_Recall_Accuracy_100features_part_{chunk_start}_{chunk_end}.xlsx", index=False)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from scipy.stats import randint

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Extract the feature values from data
X = data.values
X_test_unseen = data_test.values
# Set the chunk size (number of target variables per part)
chunk_size = 40

for chunk_start in range(200, 241, chunk_size):
    # Get the current chunk of target columns
    chunk_end = min(chunk_start + chunk_size, 241)
    target_chunk = data_target.columns[chunk_start:chunk_end]
    
    # Initialize list to store results for the current chunk
    chunk_results = []

    # Open a file to write classification reports for the current chunk
    report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Classification_Reports_100features_part_{chunk_start}_{chunk_end}.txt"
    with open(report_file_path, "w") as report_file:
        for target_col in target_chunk:
            # Extract the target column for the current category from training/validation set
            Y = data_target_train[target_col].values

            # Select top 100 features using RFE
            rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
            X_selected = rfe.fit_transform(X, Y)

            # Split the remaining training/validation set into train and validation
            X_train, X_val, Y_train, Y_val = train_test_split(X_selected, Y, test_size=0.2, random_state=42, stratify=Y)

            # Define the hyperparameter distribution
            param_dist = {
                'n_estimators': randint(100, 200),
                'max_depth': randint(20, 40),
                'min_samples_split': randint(10, 20),
                'min_samples_leaf': randint(1, 5),
                'max_features': ['sqrt', 'log2']
            }

            # Perform Randomized Search with cross-validation
            random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=60, 
                                               cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
            random_search.fit(X_train, Y_train)

            # Get the best model from Randomized Search
            best_rf = random_search.best_estimator_
            best_params = random_search.best_params_

            # Fit the model on the training data for evaluation
            best_rf.fit(X_train, Y_train)

            # Evaluate on the training set
            Y_train_pred = best_rf.predict(X_train)
            train_report = classification_report(Y_train, Y_train_pred)
            
            # Evaluate on the validation set
            Y_val_pred = best_rf.predict(X_val)
            val_report = classification_report(Y_val, Y_val_pred, output_dict=True)

            # Test on the unseen test data
            X_test_unseen_selected = rfe.transform(X_test_unseen)  # Transform unseen test set using fitted RFE
            Y_test_unseen_col = data_target_unseen[target_col].values
            Y_test_pred = best_rf.predict(X_test_unseen_selected)
            unseen_test_report = classification_report(Y_test_unseen_col, Y_test_pred, output_dict=True)

            # Write classification reports to the single file
            report_file.write(f"Classification Report for {target_col} (Train Set):\n")
            report_file.write(train_report)
            report_file.write("\n")
            report_file.write(f"Classification Report for {target_col} (Validation Set):\n")
            report_file.write(classification_report(Y_val, Y_val_pred))
            report_file.write("\n")
            report_file.write(f"Classification Report for {target_col} (Unseen Test Set):\n")
            report_file.write(classification_report(Y_test_unseen_col, Y_test_pred))
            report_file.write("\n\n" + "="*80 + "\n\n")

            # Calculate and store metrics for validation set
            precision = precision_score(Y_val, Y_val_pred, average='weighted')
            recall = recall_score(Y_val, Y_val_pred, average='weighted')
            accuracy = accuracy_score(Y_val, Y_val_pred)

            # Extract validation Class 1 metrics
            class_1_metrics = val_report.get('1', {"precision": None, "recall": None, "f1-score": None})

            # Extract unseen test Class 1 metrics
            class_1_unseen_metrics = unseen_test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

            # Calculate unseen test metrics
            unseen_precision = precision_score(Y_test_unseen_col, Y_test_pred, average='weighted')
            unseen_recall = recall_score(Y_test_unseen_col, Y_test_pred, average='weighted')
            unseen_accuracy = accuracy_score(Y_test_unseen_col, Y_test_pred)

            # Append results for the current target variable
            chunk_results.append({
                "Target Variable": target_col,
                "Validation Precision": precision,
                "Validation Recall": recall,
                "Validation Accuracy": accuracy,
                "Class 1 Precision (Validation)": class_1_metrics["precision"],
                "Class 1 Recall (Validation)": class_1_metrics["recall"],
                "Class 1 F1-Score (Validation)": class_1_metrics["f1-score"],
                "Unseen Test Precision": unseen_precision,
                "Unseen Test Recall": unseen_recall,
                "Unseen Test Accuracy": unseen_accuracy,
                "Class 1 Precision (Unseen Test)": class_1_unseen_metrics["precision"],
                "Class 1 Recall (Unseen Test)": class_1_unseen_metrics["recall"],
                "Class 1 F1-Score (Unseen Test)": class_1_unseen_metrics["f1-score"],
                "Best Hyperparameters": best_params
            })
    
    # Save the results for the current chunk to an Excel file
    results_df = pd.DataFrame(chunk_results)
    results_df.to_excel(f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Precision_Recall_Accuracy_100features_part_{chunk_start}_{chunk_end}.xlsx", index=False)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from scipy.stats import randint

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Extract the feature values from data
X = data.values
X_test_unseen = data_test.values

# Specify the single target variable
target_col = "category_stromal_34"

# Initialize a list to store results
results = []

# Set the path for the classification report file
report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Classification_Report_100features_{target_col}.txt"

# Extract the target column for the current category from training/validation set
Y = data_target_train[target_col].values

# Select top 100 features using RFE
rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
X_selected = rfe.fit_transform(X, Y)

# Split the remaining training/validation set into train and validation
X_train, X_val, Y_train, Y_val = train_test_split(X_selected, Y, test_size=0.2, random_state=42, stratify=Y)

# Define the hyperparameter distribution
param_dist = {
    'n_estimators': randint(100, 200),
    'max_depth': randint(20, 40),
    'min_samples_split': randint(10, 20),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2']
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=200, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_rf = random_search.best_estimator_
best_params = random_search.best_params_

# Fit the model on the training data for evaluation
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
train_report = classification_report(Y_train, Y_train_pred)

# Evaluate on the validation set
Y_val_pred = best_rf.predict(X_val)
val_report = classification_report(Y_val, Y_val_pred, output_dict=True)

# Test on the unseen test data
X_test_unseen_selected = rfe.transform(X_test_unseen)  # Transform unseen test set using fitted RFE
Y_test_unseen_col = data_target_unseen[target_col].values
Y_test_pred = best_rf.predict(X_test_unseen_selected)
unseen_test_report = classification_report(Y_test_unseen_col, Y_test_pred, output_dict=True)

# Write classification reports to a file
with open(report_file_path, "w") as report_file:
    report_file.write(f"Classification Report for {target_col} (Train Set):\n")
    report_file.write(train_report)
    report_file.write("\n")
    report_file.write(f"Classification Report for {target_col} (Validation Set):\n")
    report_file.write(classification_report(Y_val, Y_val_pred))
    report_file.write("\n")
    report_file.write(f"Classification Report for {target_col} (Unseen Test Set):\n")
    report_file.write(classification_report(Y_test_unseen_col, Y_test_pred))
    report_file.write("\n")

# Calculate metrics for validation set
precision = precision_score(Y_val, Y_val_pred, average='weighted')
recall = recall_score(Y_val, Y_val_pred, average='weighted')
accuracy = accuracy_score(Y_val, Y_val_pred)

# Extract validation Class 1 metrics
class_1_metrics = val_report.get('1', {"precision": None, "recall": None, "f1-score": None})

# Extract unseen test Class 1 metrics
class_1_unseen_metrics = unseen_test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

# Calculate unseen test metrics
unseen_precision = precision_score(Y_test_unseen_col, Y_test_pred, average='weighted')
unseen_recall = recall_score(Y_test_unseen_col, Y_test_pred, average='weighted')
unseen_accuracy = accuracy_score(Y_test_unseen_col, Y_test_pred)

# Append results for the target variable
results.append({
    "Target Variable": target_col,
    "Validation Precision": precision,
    "Validation Recall": recall,
    "Validation Accuracy": accuracy,
    "Class 1 Precision (Validation)": class_1_metrics["precision"],
    "Class 1 Recall (Validation)": class_1_metrics["recall"],
    "Class 1 F1-Score (Validation)": class_1_metrics["f1-score"],
    "Unseen Test Precision": unseen_precision,
    "Unseen Test Recall": unseen_recall,
    "Unseen Test Accuracy": unseen_accuracy,
    "Class 1 Precision (Unseen Test)": class_1_unseen_metrics["precision"],
    "Class 1 Recall (Unseen Test)": class_1_unseen_metrics["recall"],
    "Class 1 F1-Score (Unseen Test)": class_1_unseen_metrics["f1-score"],
    "Best Hyperparameters": best_params
})

# Save the results to an Excel file
results_df = pd.DataFrame(results)
results_df.to_excel(f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Precision_Recall_Accuracy_100features_{target_col}.xlsx", index=False)


Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [9]:
# import numpy as np
# import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_selection import RFE
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
# from scipy.stats import randint

# # Initialize the Random Forest Classifier
# rf = RandomForestClassifier(random_state=42)

# # Extract the feature values from data
# X = data.values

# # Initialize lists to store precision, recall, and accuracy for each target variable
# results = []

# # Open a single file to write all classification reports
# with open("/home/qiuaodon/Desktop/CRC_image/Best_features_REF/Classification_Reports_100features.txt", "w") as report_file:
    
#     # Loop through the first 120 target columns in data_target
#     for i, target_col in enumerate(data_target.columns[:241]):
#         # Extract the target column for the current category
#         Y = data_target[target_col].values

#         # Select top 100 features using RFE
#         rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
#         X_selected = rfe.fit_transform(X, Y)
        
#         # Split the data into training and testing sets
#         X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size=0.2, random_state=42)

#         # Define the hyperparameter distribution (use distributions for random search)
#         param_dist = {
#             'n_estimators': randint(100, 200),
#             'max_depth': randint(20, 40),
#             'min_samples_split': randint(10, 20),
#             'min_samples_leaf': randint(1, 5),
#             'max_features': ['sqrt', 'log2']
#         }

#         # Perform Randomized Search with cross-validation
#         random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=60, 
#                                            cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
#         random_search.fit(X_train, Y_train)

#         # Get the best model from Randomized Search
#         best_rf = random_search.best_estimator_
#         best_params = random_search.best_params_

#         # Fit the model on the training data for the final evaluation on train and test sets
#         best_rf.fit(X_train, Y_train)

#         # Evaluate on the training set
#         Y_train_pred = best_rf.predict(X_train)
#         train_report = classification_report(Y_train, Y_train_pred)
        
#         # Evaluate on the test set
#         Y_test_pred = best_rf.predict(X_test)
#         test_report = classification_report(Y_test, Y_test_pred, output_dict=True)

#         # Write classification reports to the single file
#         report_file.write(f"Classification Report for {target_col} (Train Set):\n")
#         report_file.write(train_report)
#         report_file.write("\n")
#         report_file.write(f"Classification Report for {target_col} (Test Set):\n")
#         report_file.write(classification_report(Y_test, Y_test_pred))
#         report_file.write("\n\n" + "="*80 + "\n\n")

#         # Calculate and store precision, recall, and accuracy
#         precision = precision_score(Y_test, Y_test_pred, average='weighted')
#         recall = recall_score(Y_test, Y_test_pred, average='weighted')
#         accuracy = accuracy_score(Y_test, Y_test_pred)

#         # Extract precision, recall, and f1-score specifically for class '1'
#         class_1_metrics = test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

#         results.append({
#             "Target Variable": target_col,
#             "Precision": precision,
#             "Recall": recall,
#             "Accuracy": accuracy,
#             "Class 1 Precision": class_1_metrics["precision"],
#             "Class 1 Recall": class_1_metrics["recall"],
#             "Class 1 F1-Score": class_1_metrics["f1-score"],
#             "Best Hyperparameters": best_params
#         })

# # Save the results to an Excel file
# results_df = pd.DataFrame(results)
# results_df.to_excel("/home/qiuaodon/Desktop/CRC_image/Best_features_REF/Precision_Recall_Accuracy_100features.xlsx", index=False)


Fitting 5 folds for each of 60 candidates, totalling 300 fits


KeyboardInterrupt: 

# combine the results

In [5]:
import pandas as pd
import glob

# Specify the directory where your files are located
file_path_pattern = '/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Precision_Recall_Accuracy_100features_part_*.xlsx'

# Use glob to find all matching files
all_files = glob.glob(file_path_pattern)

# Create an empty list to store DataFrames
data_frames = []

# Loop through each file and append its DataFrame to the list
for file in all_files:
    df = pd.read_excel(file)
    data_frames.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(data_frames, ignore_index=True)

# Save the combined DataFrame to a new Excel file
combined_df.to_excel('/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/top_100features_Randomforest_241_targets_90train.xlsx', index=False)
combined_df


,Target Variable,Validation Precision,Validation Recall,Validation Accuracy,Class 1 Precision (Validation),Class 1 Recall (Validation),Class 1 F1-Score (Validation),Unseen Test Precision,Unseen Test Recall,Unseen Test Accuracy,Class 1 Precision (Unseen Test),Class 1 Recall (Unseen Test),Class 1 F1-Score (Unseen Test),Best Hyperparameters
0,category_tnk_1,0.476098,0.487805,0.487805,0.535714,0.555556,0.545455,0.371849,0.357143,0.357143,0.294118,0.357143,0.322581,"{'max_depth': 25, 'max_features': 'log2', 'min..."
1,category_tnk_2,0.514563,0.512195,0.512195,0.483871,0.555556,0.517241,0.543515,0.404762,0.404762,0.375000,0.400000,0.387097,"{'max_depth': 23, 'max_features': 'sqrt', 'min..."
2,category_tnk_3,0.483284,0.487805,0.487805,0.448276,0.464286,0.456140,0.180934,0.166667,0.166667,0.125000,0.181818,0.148148,"{'max_depth': 37, 'max_features': 'log2', 'min..."
3,category_tnk_4,0.560694,0.548780,0.548780,0.650000,0.500000,0.565217,0.588409,0.547619,0.547619,0.733333,0.523810,0.611111,"{'max_depth': 28, 'max_features': 'sqrt', 'min..."
4,category_tnk_5,0.546459,0.548780,0.548780,0.560000,0.518519,0.538462,0.353672,0.309524,0.309524,0.266667,0.285714,0.275862,"{'max_depth': 36, 'max_features': 'sqrt', 'min..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,category_stromal_10,0.456924,0.451220,0.451220,0.451613,0.500000,0.474576,0.480866,0.428571,0.428571,0.315789,0.600000,0.413793,"{'max_depth': 30, 'max_features': 'log2', 'min..."
237,category_stromal_11,0.501768,0.500000,0.500000,0.523810,0.407407,0.458333,0.367565,0.333333,0.333333,0.363636,0.266667,0.307692,"{'max_depth': 39, 'max_features': 'sqrt', 'min..."
238,category_stromal_12,0.426071,0.426829,0.426829,0.379310,0.407407,0.392857,0.379085,0.404762,0.404762,0.500000,0.500000,0.500000,"{'max_depth': 31, 'max_features': 'log2', 'min..."
239,category_stromal_13,0.565932,0.560976,0.560976,0.636364,0.500000,0.560000,0.391484,0.357143,0.357143,0.250000,0.363636,0.296296,"{'max_depth': 22, 'max_features': 'sqrt', 'min..."
